<a href="https://colab.research.google.com/github/KomaliValluru/DS/blob/main/mnist22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The MNIST database of handwritten digits.

label
call_split
bar_chart
70,000 items
Value
arrow_drop_up
Count
0
6,903
1
7,877
2
6,990
3
7,141
4
6,824
5
6,313
6
6,876
7
7,293
8
6,825
9
6,958
split
call_split
bar_chart
70,000 items
Value
arrow_drop_up
Count
test
10,000
train


In [7]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# You cannot modify from here until it is indicated by a comment
(test_data),test_data_info=tfds.load('mnist',split='test',with_info=True,as_supervised=True)

(train_data),ds_info=tfds.load('mnist',split=['train[10000:45000]'],with_info=True,as_supervised=True)

def getnewtst():
  (new_test),new_test_info=tfds.load('mnist',split=['train[0:9999]'],with_info=True,as_supervised=True)
  new_test = new_test[0].map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
  new_test = new_test.batch(64)
  new_test = new_test.cache()
  new_test = new_test.prefetch(tf.data.AUTOTUNE)
  return new_test


# Can modify code now below this comment

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`.
  The model wants the float and tfds gives you 0-255."""
  return tf.cast(image, tf.float32) / 255., label


train_data = train_data[0].map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.cache()
train_data = train_data.shuffle(ds_info.splits['train'].num_examples)
train_data = train_data.batch(64)
train_data = train_data.prefetch(tf.data.AUTOTUNE)


test_data = test_data.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(128)
test_data = test_data.cache()
test_data = test_data.prefetch(tf.data.AUTOTUNE)



In [28]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 ),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=2, padding='same',
                 activation='relu'
                 ),
tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, activation='softmax')

])
model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.3,learning_rate=0.08),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    train_data,
    epochs=7,
    validation_data=test_data,
)

Epoch 1/7
547/547 [==============================] - 43s 75ms/step - loss: 0.1559 - sparse_categorical_accuracy: 0.9563 - val_loss: 0.0994 - val_sparse_categorical_accuracy: 0.9741
Epoch 2/7
547/547 [==============================] - 40s 72ms/step - loss: 0.0614 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0540 - val_sparse_categorical_accuracy: 0.9821
Epoch 3/7
547/547 [==============================] - 40s 72ms/step - loss: 0.0443 - sparse_categorical_accuracy: 0.9868 - val_loss: 0.0433 - val_sparse_categorical_accuracy: 0.9855
Epoch 4/7
547/547 [==============================] - 40s 73ms/step - loss: 0.0342 - sparse_categorical_accuracy: 0.9898 - val_loss: 0.0545 - val_sparse_categorical_accuracy: 0.9819
Epoch 5/7
547/547 [==============================] - 40s 73ms/step - loss: 0.0264 - sparse_categorical_accuracy: 0.9928 - val_loss: 0.0457 - val_sparse_categorical_accuracy: 0.9841
Epoch 6/7
547/547 [==============================] - 41s 76ms/step - loss: 0.0190 - sparse_cate

In [29]:

results = model.evaluate(getnewtst())
print("test loss, test acc:", results)


157/157 [==============================] - 4s 27ms/step - loss: 0.0512 - sparse_categorical_accuracy: 0.9866
test loss, test acc: [0.05123588442802429, 0.9865986704826355]
